In [1]:
def yolo_to_corners(label, img_width, img_height):
    """
    Преобразует лейбл YOLO в координаты углов прямоугольника.

    :param label: Лейбл YOLO в формате [class_id, x_center, y_center, width, height]
    :param img_width: Ширина изображения в пикселях
    :param img_height: Высота изображения в пикселях
    :return: Кортеж с координатами углов прямоугольника (x_min, y_min, x_max, y_max)
    """
    class_id, x_center_norm, y_center_norm, width_norm, height_norm = label

    # Денормализация координат+

    x_center = x_center_norm * img_width
    y_center = y_center_norm * img_height
    width = width_norm * img_width
    height = height_norm * img_height

    # Вычисление координат углов
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return x_min, y_min, x_max, y_max

In [2]:
import os 
import cv2 as cv
import shutil

In [3]:
path_img_train = "F:/datasets/images/train/"
path_label_train = "F:/datasets/labels/train/"

path_img_test = "F:/datasets/images/val/"
path_label_test = "F:/datasets/labels/val/"

files_img = sorted(os.listdir(path_img_train))
files_label = sorted(os.listdir(path_img_train))

In [5]:
class BreakLoops(Exception):
    pass

try:
    for i, filename in enumerate(files_img):
        img = cv.imread(path_img_train + filename)
        h, w = img.shape[:2]
        label_list = []

        with open(path_label_train + filename.replace(".jpg", ".txt"), 'r') as f:
            for line in f.readlines():
                label, x1, y1, x2, y2 = line.split()
                label_list.append([int(label), float(x1), float(y1), float(x2), float(y2)])

        
        for label in label_list:
            x_min, y_min, x_max, y_max = yolo_to_corners(label, w, h)
            print(x_min, y_min, x_max, y_max)
            cv.rectangle(img, (x_min, y_min),(x_max, y_max), (255,0,0),3)

        cv.imshow("image", img)


        while True:
            key = cv.waitKey(0) & 0xFF
            if key == ord('e'):
                print(f"Продолжение цикла. {i}")
                shutil.move(path_img_train + filename, "F:/datasets/images_corrent/train/" + filename)
                shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_corrent/train/" + filename.replace(".jpg", ".txt"))
                break
            elif key == ord('w'):  
                print(f"Неправильная разметка. {i}")
                shutil.move(path_img_train + filename, "F:/datasets/images_incorrent/train/" + filename)
                shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_incorrent/train/" + filename.replace(".jpg", ".txt"))
                break
            elif key == ord('q'):
                raise BreakLoops     
except BreakLoops:
    pass

cv.destroyAllWindows()

27 407 82 496
111 250 141 284
194 244 206 264
207 242 226 258
220 272 243 304
360 296 377 316
460 372 498 405
522 244 529 256
558 285 581 307
570 534 624 639
606 260 627 303
611 308 633 343
622 279 635 300
658 281 672 303
676 278 707 301
681 297 713 350
722 295 744 332
773 344 795 363
820 280 842 304
850 343 882 388
888 313 911 345
892 396 915 421
969 344 995 379
988 434 1000 455


In [6]:
ds = 'F:/datasets/'
ln_1="#Paths \n"
ln_2='train: ' +"'"+ path_img_train +"' \n"
ln_3='val: ' +"'" + path_img_test +"' \n"
ln_4='test: ' +"'" + path_img_test +"' \n"
ln_5="\n"
ln_6='# Classes\n'
ln_7='names:\n'
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

with open("config.yaml", 'w') as f:
    f.writelines(config_lines)

In [7]:
from ultralytics import YOLO

In [9]:
model = YOLO('yolo11m.yaml').load('yolo11m.pt')

100%|██████████| 38.8M/38.8M [02:50<00:00, 239kB/s]


Transferred 649/649 items from pretrained weights


In [10]:
model.train(
    data='F:\datasets\config.yaml',
    epochs=50,
    batch=8,
    name='model_1',
    device=0,
    save=True           
)

New https://pypi.org/project/ultralytics/8.3.73 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.yaml, data=F:\datasets\config.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=model_120, exist_ok=False, pretrained=yolo11m.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning F:\datasets\labels\train... 13136 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13136/13136 [00:06<00:00, 1982.91it/s]


train: New cache created: F:\datasets\labels\train.cache


val: Scanning F:\datasets\labels\val.cache... 3347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]


Plotting labels to runs\detect\model_120\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\model_120
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       3347      10299      0.884      0.771      0.854      0.554

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/50      4.73G      1.223     0.7492      1.134         33        640: 100%|██████████| 1642/1642 [13:29<00:00,  2.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 210/210 [01:15<00:00,  2.77it/s]


                   all       3347      10299      0.886      0.787      0.862      0.564

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/50      4.72G      1.193     0.7184      1.124         16        640:  45%|████▌     | 747/1642 [06:11<07:25,  2.01it/s]


KeyboardInterrupt: 

In [17]:
import requests

files = {'file': open('123.png', 'rb')}
r = requests.post("http://localhost:8000/answer", files=files)

In [20]:
r.json()

'answer: [[243.02847290039062, 357.5403747558594, 447.7892761230469, 587.488037109375, 0.8693018555641174, 0.0]]'